In [ ]:
!pip install qiskit
!pip install qiskit[visualization]
!pip install 'qiskit-aqua[cplex]'
!pip install 'qiskit-optimization[cplex]'
!pip install 'qiskit-aer-gpu'
from qiskit import QuantumRegister, ClassicalRegister, Aer, BasicAer, QuantumCircuit, execute
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator,StatevectorSimulator
from qiskit.visualization import plot_state_city,circuit_drawer
from qiskit.quantum_info import Statevector
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 9.0 MB/s 
     |████████████████████████████████| 19.2 MB 1.2 MB/s 
     |████████████████████████████████| 240 kB 38.6 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 112 kB 46.3 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 943 kB 47.0 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 4.0 MB 40.8 MB/s 
     |████████████████████████████████| 1.9 MB 48.7 MB/s 
     |████████████████████████████████| 112 kB 59.7 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.0-py3-none-any.whl size=12188 sha256=860e0d7d21667034ee77297a0cbb74d31346ed5c739c77c3a265d8c46f785382
  Stored in directory: /root/.cache/pip/wheels/63/d0/9a/25714

### **Grover's Oracle and Grovers Diffusion Source Code** 

***taken from https://arxiv.org/abs/1903.04359***

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, Aer, execute
import numpy as np
import math as m
import scipy as sci

S_simulator = Aer.backends(name='statevector_simulator')[0]
M_simulator = Aer.backends(name='qasm_simulator')[0] 

#Displaying Results 
def Wavefunction( obj , *args, **kwargs):
#Displays the waveftmction of the quantum system 
	if(type(obj) == QuantumCircuit ):
		statevec = execute( obj, S_simulator, shots=1 ).result().get_statevector()
	if(type(obj) == np.ndarray):
		statevec = obj
	sys = False
	NL = False
	dec = 5
	if 'precision' in kwargs:
		dec = int( kwargs['precision'] )
	if 'column' in kwargs:
		NL = kwargs['column']
	if 'systems' in kwargs:
		systems = kwargs['systems']
		sys = True
		last_sys = int(len(systems)-1)
		show_systems = []
		for s_chk in np.arange(len(systems)):
			if( type(systems[s_chk]) != int ):
				raise Exception('systems must be an array of all integers')
		if 'show_systems' in kwargs:
			show_systems = kwargs['show_systems']
			if( len(systems)!= len(show_systems) ):
				raise Exception('systems and show_systems need to be arrays of equal length')
			for ls in np.arange(len(show_systems)):
				if((show_systems[ls] != True) and (show_systems[ls] != False)):
					raise Exception('show_systems must be an array of Truth Values')
				if(show_systems[ls] == True):
					last_sys = int(ls) Copy of QuantumDifferentialAttack_old
		else:
			for ss in np.arange(len(systems)):
				show_systems.append(True)
	wavefunction = ''
	qubits = int(m.log(len(statevec),2))
	for i in np.arange( int(len(statevec))):
		#print(wavefunction)
		value = round(statevec[i].real, dec) + round(statevec[i].imag, dec) * 1j
		if( (value.real != 0) or (value.imag != 0)):
			state = list(Binary(int(i),int(2**qubits)))
			state.reverse()
			state_str = ''
			#print(state)
			if( sys == True ): #Systems and SharSystems 
				k = 0 
				for s in np.arange(len(systems)):
					if(show_systems[s] == True):
						if(int(s) != last_sys):
							state.insert(int(k + systems[s]), '>|' )
							k = int(k + systems[s] + 1)
						else:
							k = int(k + systems[s]) 
					else:
						for s2 in np.arange(systems[s]):
							del state[int(k)]
			for j in np.arange(len(state)):
				if(type(state[j])!= str):
					state_str = state_str + str(int(state[j])) 
				else:
					state_str = state_str + state[j]
			#print(state_str)
			#print(value)
			if( (value.real != 0) and (value.imag != 0) ):
				if( value.imag > 0):
					wavefunction = wavefunction + str(value.real) + '+' + str(value.imag) + 'j |' + state_str + '>   '
				else:
					wavefunction = wavefunction + str(value.real) + '' + str(value.imag) + 'j |' + state_str +  '>   '
			if( (value.real !=0 ) and (value.imag ==0) ):
				wavefunction = wavefunction  + str(value.real) + '  |' + state_str + '>   '
			if( (value.real == 0) and (value.imag != 0) ):
				wavefunction = wavefunction + str(value.imag)  + 'j |' + state_str + '>   '
			if(NL):
				wavefunction = wavefunction + '\n'
		#print(NL)
	
	#print(wavefunction)
	return wavefunction


def Measurement(quantumcircuit, *args, **kwargs): 
	#Displays the measurement results of a quantum circuit 
	p_M = True
	S = 1
	ref = False
	NL = False
	if 'shots' in kwargs:
		S = int(kwargs['shots'])
	if 'return_M' in kwargs:
		ret = kwargs['return_M']
	if 'print_M' in kwargs:
		p_M = kwargs['print_M']
	if 'column' in kwargs:
		NL = kwargs['column']
	M1 = execute(quantumcircuit, M_simulator, shots=S).result().get_counts(quantumcircuit)
	M2 = {}
	k1 = list(M1.keys())
	v1 = list(M1.values())
	for k in np.arange(len(k1)):
		key_list = list(k1[k])
		new_key = ''
		for j in np.arange(len(key_list)):
			new_key = new_key+key_list[len(key_list)-(j+1)]
		M2[new_key] = v1[k]
	if(p_M):
		k2 = list(M2.keys())
		v2 = list(M2.values())
		measurements = ''
		for i in np.arange(len(k2)):
			m_str = str(v2[i])+'|'
			for j in np.arange(len(k2[i])):
				if(k2[i][j] == '0'):
					m_str = m_str + '0' 
				if(k2[i][j] == '1'):
					m_str = m_str + '1'
				if( k2[i][j] == ' ' ):
					m_str = m_str +'>|'
			m_str = m_str + '>   '
			if(NL):
				m_str = m_str + '\n'
			measurements = measurements + m_str
		#print(measurements)
		return measurements
	if(ref):
		return M2


#Math Operations
def Oplus(bit1,bit2): 
	'''Adds too bits of O's and 1's (modulo 2)'''
	bit = np.zeros(len(bit1))
	for i in np.arange( len(bit) ):
		if( (bit1[i]+bit2[i])%2 == 0 ):
			bit[i] = 0
		else: 
			bit[i] = 1
	return bit 


def Binary(number,total): 
#Converts a number to binary, right to left LSB 152 153 o
	qubits = int(m.log(total,2))
	N = number
	b_num = np.zeros(qubits)
	for i in np.arange(qubits):
		if( N/((2)**(qubits-i-1)) >= 1 ):
			b_num[i] = 1
			N = N - 2 ** (qubits-i-1)
	B = [] 
	for j in np.arange(len(b_num)):
		B.append(int(b_num[j]))
	return B

def From_Binary(s):
    num = 0
    for i in np.arange(len(s)):
        num = num + s[int(0-(i+1))] * 2 ** (i)
    return num

def B2D(in_bi):
    len_in = len(in_bi)
    in_bi = in_bi[::-1]
    dec = 0
    for i in range(0,len_in):
        if in_bi[i] != '0':
            dec += 2**i
    return dec

#  Custom Gates
def x_Transformation(qc, qreg, state): 
#Tranforms the state of the system, applying X gates according to as in the vector 'state' 
	for j in np.arange(len(state)):
		if( int(state[j]) == 0 ):
			qc.x( qreg[int(j)] ) 


def n_NOT(qc, control, target, anc): 
#performs an n-NOT gate
	n = len(control)
	instructions = []
	active_ancilla = []
	q_unused = []
	q = 0
	a = 0
	while( (n > 0) or (len(q_unused) != 0) or (len(active_ancilla) != 0) ):
		if( n > 0 ):
			if( (n-2) >= 0 ):
				instructions.append( [control[q], control[q+1], anc[a]] )
				active_ancilla.append(a)
				a += 1
				q += 2
				n = n - 2
			if( (n-2) == -1 ):
				q_unused.append( q )
				n = n - 1
		elif( len(q_unused) != 0 ):
			if(len(active_ancilla)!=1):
				instructions.append( [control[q], anc[active_ancilla[0]], anc[a]] )
				del active_ancilla[0]
				del q_unused[0]
				active_ancilla.append(a)
				a = a + 1
			else:
				instructions.append( [control[q], anc[active_ancilla[0]], target] )
				del active_ancilla[0]
				del q_unused[0]
		elif( len(active_ancilla) != 0 ):
			if( len(active_ancilla) > 2 ):
				instructions.append( [anc[active_ancilla[0]], anc[active_ancilla[1]], anc[a]] )
				active_ancilla.append(a)
				del active_ancilla[0]
				del active_ancilla[0]
				a = a + 1
			elif( len(active_ancilla) == 2):
				instructions.append([anc[active_ancilla[0]], anc[active_ancilla[1]], target])
				del active_ancilla[0]
				del active_ancilla[0]
	for i in np.arange( len(instructions) ):
		qc.ccx( instructions[i][0], instructions[i][1], instructions[i][2] )
	del instructions[-1]
	for i in np.arange( len(instructions) ):
		qc.ccx( instructions[0-(i+1)][0], instructions[0-(i+1)][1], instructions[0-(i+1)][2] )



def Control_Instruction( qc, vec ): 
#Ammends the proper quantum circuit instruction based on the input 'vec'
#Used for the function 'n_Control_U
	if( vec[0] == 'X' ):
		qc.cx( ver[1], vec[2] )
	if( vec[0] == 'Z' ):
		qc.cz( ver[1], vec[2] )
	if( vec[0] == 'PRASE' ):
		qc.cu1( vec[2], vec[1], vec[3] )
	if( vec[0] == 'SWAP' ):
		qc.cswap( vec[1], vec[2], vec[3] ) 

def X_Transformation(qc, qreg, state):
	for j in np.arange(len(state)):
		if( int(state[j]) == 0):
			qc.x( qreg[int(j)])



def sinmons_solver(E,N):
	'''Returns an array of s_prime candidates
	'''
	s_primes = []
	for s in np.ararge(1,2**N):
		sp = Binary( int(s), 2**N )
		candidate = True
		for e in np.arange( len(E) ):
			value = 0
			for i in np.arange( N ):
				value = value + sp[i]*E[e][i]
			if(value%2==1):
				candidate=False
		if(candidate):
			s_primes.append(sp)
	return s_primes


def Grover_Oracle(mark, qc, q, an1, an2): 
	'''
	picks out the marked state and applies a negative phase 
	'''
	qc.h( an1[0] )
	X_Transformation(qc, q, mark)
	if( len(mark) > 2 ):
		n_NOT( qc, q, an1[0], an2 )
	if( len(mark) == 2 ):
		qc.ccx( q[0], q[1], an1[0] )
	X_Transformation(qc, q, mark)
	qc.h( an1[0] )

def Grover_Diffusion(mark, qc, q, an1, an2): 
	'''
	ammends the instructions for a Grover Diffusion Operation to the Quartu rcuit
	'''
	zeros_state = []
	for i in np.arange( len(mark) ):
		zeros_state.append( 0 )
		qc.h( q[int(i)] )
	Grover_Oracle(zeros_state, qc, q, an1, an2)
	for j in np.arange( len(mark) ):
		qc.h( q[int(j)] )



def Grover(Q, marked): 
	'''
	Amends all the instructions for a Grover Search 
	'''
	q = QuantumRegister(Q,name='q')
	an1 = QuantumRegister(1,name='anc')
	an2 = QuantumRegister(Q-2,name='nanc')
	c = ClassicalRegister(Q,name='c')
	qc = QuantumCircuit(q,an1,an2,c,name='qc')
	for j in np.arange(Q):
		qc.h( q[int(j)] )
	qc.x( an1[0] )
	iterations = round( m.pi/4 * 2**(Q/2.0) )
	for i in np.arange( iterations ):
		Grover_Oracle(marked, qc, q, an1, an2)
		Grover_Diffusion(marked, qc, q, an1, an2)
	return qc, q, an1, an2, c


### **Encryption and Decryption Functions of smallGIFT**

In [ ]:
def Encryption(circuit,rounds,start,kreg):
  for i in range(0,rounds):
    #############First S-box#################
    circuit.ccx(qreg[0+start], qreg[2+start], qreg[1+start])
    circuit.ccx(qreg[3+start], qreg[1+start], qreg[0+start])
    circuit.x(qreg[0+start])
    circuit.x(qreg[1+start])
    circuit.ccx(qreg[0+start], qreg[1+start], qreg[2+start])
    circuit.x(qreg[0+start])
    circuit.x(qreg[1+start])
    circuit.x(qreg[2+start])
    circuit.cx(qreg[2+start], qreg[3+start])
    circuit.x(qreg[3+start])
    circuit.cx(qreg[3+start], qreg[1+start])
    circuit.x(qreg[1+start])
    circuit.ccx(qreg[0+start], qreg[1+start], qreg[2+start])
    circuit.swap(qreg[0+start],qreg[3+start])

    #############Second S-box#################
    circuit.ccx(qreg[4+0+start], qreg[4+2+start], qreg[4+1+start])
    circuit.ccx(qreg[4+3+start], qreg[4+1+start], qreg[4+0+start])
    circuit.x(qreg[4+0+start])
    circuit.x(qreg[4+1+start])
    circuit.ccx(qreg[4+0+start], qreg[4+1+start], qreg[4+2+start])
    circuit.x(qreg[4+0+start])
    circuit.x(qreg[4+1+start])
    circuit.x(qreg[4+2+start])
    circuit.cx(qreg[4+2+start], qreg[4+3+start])
    circuit.x(qreg[4+3+start])
    circuit.cx(qreg[4+3+start], qreg[4+1+start])
    circuit.x(qreg[4+1+start])
    circuit.ccx(qreg[4+0+start], qreg[4+1+start], qreg[4+2+start])
    circuit.swap(qreg[4+0+start],qreg[4+3+start])

    #############Permutation#################
    circuit.swap(qreg[1+start], qreg[3+start])
    circuit.swap(qreg[1+start], qreg[7+start])
    circuit.swap(qreg[2+start], qreg[5+start])
    circuit.swap(qreg[2+start], qreg[4+start])

    #############Key Addition#################
    circuit.cx(kreg[0],qreg[0+start])
    circuit.cx(kreg[1],qreg[4+start])
    circuit.cx(kreg[2],qreg[1+start])
    circuit.cx(kreg[3],qreg[5+start])
def Decryption(circuit,rounds,start,kreg):
  for i in range(0,rounds):
    #############Key Addition#################
    circuit.cx(kreg[3],qreg[5+start])
    circuit.cx(kreg[2],qreg[1+start])
    circuit.cx(kreg[1],qreg[4+start])
    circuit.cx(kreg[0],qreg[0+start])

    #############Permutation#################
    circuit.swap(qreg[2+start], qreg[4+start])
    circuit.swap(qreg[2+start], qreg[5+start])
    circuit.swap(qreg[1+start], qreg[7+start])
    circuit.swap(qreg[1+start], qreg[3+start])

    #############First S-box#################
    circuit.swap(qreg[0+start],qreg[3+start])
    circuit.ccx(qreg[0+start], qreg[1+start], qreg[2+start])
    circuit.x(qreg[1+start])
    circuit.cx(qreg[3+start], qreg[1+start])
    circuit.x(qreg[3+start])
    circuit.cx(qreg[2+start], qreg[3+start])
    circuit.x(qreg[2+start])
    circuit.x(qreg[1+start])
    circuit.x(qreg[0+start])
    circuit.ccx(qreg[0+start], qreg[1+start], qreg[2+start])
    circuit.x(qreg[1+start])
    circuit.x(qreg[0+start])
    circuit.ccx(qreg[3+start], qreg[1+start], qreg[0+start])
    circuit.ccx(qreg[0+start], qreg[2+start], qreg[1+start])

    #############SecondS-box#################
    circuit.swap(qreg[4+0+start],qreg[4+3+start])
    circuit.ccx(qreg[4+0+start], qreg[4+1+start], qreg[4+2+start])
    circuit.x(qreg[4+1+start])
    circuit.cx(qreg[4+3+start], qreg[4+1+start])
    circuit.x(qreg[4+3+start])
    circuit.cx(qreg[4+2+start], qreg[4+3+start])
    circuit.x(qreg[4+2+start])
    circuit.x(qreg[4+1+start])
    circuit.x(qreg[4+0+start])
    circuit.ccx(qreg[4+0+start], qreg[4+1+start], qreg[4+2+start])
    circuit.x(qreg[4+1+start])
    circuit.x(qreg[4+0+start])
    circuit.ccx(qreg[4+3+start], qreg[4+1+start], qreg[4+0+start])
    circuit.ccx(qreg[4+0+start], qreg[4+2+start], qreg[4+1+start])

### **Initialization of Encryption and Decryption Functions for Key Recovery Attack**

Parameters:
1. **circuit**- *Quantum Circuit*
2. **rounds** - *No. of rounds*
3. **block_size**- *It is used to encrypt/decrypt second(q_8 to q_15) plaintext/ciphertext*
4. **end_diff** - *Whether the output is difference(xor) of two plaintext or not, if true the output qubits q_8 to q_15 will contain the difference q_0...q_7 xor q_8...q15*
5. **kreg**- *qubit of keys*

In [ ]:
def Initialize_Encryption(circuit, rounds, block_size, end_diff,kreg):

  if (rounds == 1):
    for i in range(block_size):
        circuit.cx(qreg[i],qreg[i+block_size])
  if (rounds > 1): 
    for i in range(block_size):
          circuit.cx(qreg[i],qreg[i+block_size])

          
    for i in range(block_size):
            if(input_diff[i]==1):
                circuit.x(qreg[2*block_size-1-i])

  Encryption(circuit,rounds,0,kreg)
  Encryption(circuit,rounds,block_size,kreg)

  if (end_diff == True and rounds >1):
    for i in range(block_size):
        circuit.cx(qreg[i],qreg[i+block_size])

def Initialize_Decryption(circuit, rounds, block_size, end_diff,kreg):
  if (end_diff == True and rounds >1):
    for i in range(block_size):
        circuit.cx(qreg[i],qreg[i+block_size])

  Decryption(circuit,rounds,0,kreg)
  Decryption(circuit,rounds,block_size,kreg)

  if (rounds > 1):
    for i in range(block_size):
          if(input_diff[i]==1):
              circuit.x(qreg[2*block_size-1-i])
    for i in range(block_size):
        circuit.cx(qreg[i],qreg[i+block_size])

  if (end_diff == True and rounds ==1):
    for i in range(block_size):
        circuit.cx(qreg[i],qreg[i+block_size])

### **Source code for key recovery attack.** 
The code is executed for all possilbe 16 keys and the results indicate that in each case key is successfully recovered (highest counter corresponding to correct key)

In [ ]:
for key in range(0,16):
  key_bin = bin(key)[2:].zfill(4)
  print('Key is: ',key_bin)
  counter = [0]*16
  counter_top = [0]*16
  block_size = 8
  key_size = 4
  rounds = 3                                  # rounds = 4 if filter condition after 4 rounds (for the difference 0x09 after 3 rounds)
  qreg = QuantumRegister(block_size*2, 'q')
  kreg_o = QuantumRegister(key_size, 'k_o')
  kreg_n = QuantumRegister(key_size, 'k_n')
  
  an1 = QuantumRegister(1,name='anc')
  an2 = QuantumRegister(block_size-2,name='nanc')
  c = ClassicalRegister(block_size*2 + key_size,name='c')

  circuit = QuantumCircuit(qreg,kreg_o,kreg_n,an1,an2,c,name='qc')
  simulator = QasmSimulator(device="GPU") 
  input_diff = [0,0,0,0,0,0,0,1]
  output_diff = [0,0,1,0,1,0,0,1]
  #output_diff = [0,0,0,0,1,0,0,1]   # for the output difference 0x09 after 3 rounds
  #output_diff_nr = [0,0,0,0]        # filter condition after 4 rounds (for the difference 0x09 after 3 rounds)

  for i in np.arange(block_size):
      circuit.h(qreg[i] )

  circuit.x(an1[0])

  if(key_bin[3]=='1'):
    circuit.x(kreg_o[0])
  if(key_bin[2]=='1'):
    circuit.x(kreg_o[1])
  if(key_bin[1]=='1'):
    circuit.x(kreg_o[2])
  if(key_bin[0]=='1'):
    circuit.x(kreg_o[3])

  for i in range(0,13):
    Initialize_Encryption(circuit, rounds, block_size,True,kreg_o)
    #Grover_Oracle(output_diff_nr, circuit, [qreg[9],qreg[10],qreg[12],qreg[14]], an1, an2)  #uncomment if filter condition after 4 rounds (for the difference 0x09 after 3 rounds)
    Grover_Oracle(output_diff, circuit, qreg[15:7:-1], an1, an2)                             #comment if filter condition after 4 rounds (for the difference 0x09 after 3 rounds) 
    Initialize_Decryption(circuit, rounds, block_size,True,kreg_o)
    Grover_Diffusion([0]*block_size, circuit, qreg[0:8], an1, an2)

  for i in range(0,block_size*2):
    circuit.measure(qreg[i],c[i])

  job = execute(circuit, simulator, shots=1024)
  result = job.result()
  counts = result.get_counts(circuit)
  sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
  print(sorted_counts)                                                                      # Right pairs with sorted counters/probability 
  fix = [k[12:] for k, v in sorted_counts.items() if v >= 10]                               # >=10 condition can be adjusted to include number of right pairs, it should be an experimental value

  for fix_q in fix: 
    block_size = 8
    key_size = 4
    rounds = 4
    qreg = QuantumRegister(block_size*2, 'q')
    kreg_o = QuantumRegister(key_size, 'k_o')
    kreg_n = QuantumRegister(key_size, 'k_n')
    
    an1 = QuantumRegister(1,name='anc')
    an2 = QuantumRegister(block_size-2,name='nanc')
    c = ClassicalRegister(block_size*2 + key_size,name='c')

    circuit = QuantumCircuit(qreg,kreg_o,kreg_n,an1,an2,c,name='qc')
    simulator = QasmSimulator(device="GPU") 

    for i in range(0,len(fix_q)):
      if(fix_q[i]=='1'):
        circuit.x(qreg[block_size-1-i])    

    circuit.x(an1[0])

    if(key_bin[3]=='1'):
      circuit.x(kreg_o[0])
    if(key_bin[2]=='1'):
      circuit.x(kreg_o[1]) 
    if(key_bin[1]=='1'):
      circuit.x(kreg_o[2])
    if(key_bin[0]=='1'):
      circuit.x(kreg_o[3])

    Initialize_Encryption(circuit, rounds, block_size,False,kreg_o)

    for i in range(0,key_size):
      circuit.h(kreg_n[i]) 

    for i in range(0,3):
      Initialize_Decryption(circuit, 1, block_size,True,kreg_n)
      Grover_Oracle(output_diff, circuit, qreg[15:7:-1], an1, an2)
      Initialize_Encryption(circuit, 1, block_size,False,kreg_n) 
      Grover_Diffusion([0]*key_size, circuit, kreg_n, an1, an2)

    for i in range(0,key_size):
      circuit.measure(kreg_n[i],c[i + (block_size*2)])
    job = execute(circuit, simulator, shots=1024) 
    result = job.result()
    counts = result.get_counts(circuit)
    sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
    print('Sorted Key Counts:',sorted_counts)                                    # Correct keys with sorted counters/probability for each right pair
    counter_top[int(list(sorted_counts)[0][0:4],2)] += 1
    for (k,v) in sorted_counts.items():
      int_k = int(k[0:4],2)
      counter[int_k] += v
    print(counter)                                                                # Correct keys with sorted counters/probability for right pairs (commulative)
    

  print('Key is: ',key_bin) 
  print("Total Key Counter")
  for i in range(0,2**(key_size)):
    print(str(bin(i))[2:].zfill(4) + " : " + str(counter[i]) )                    # Correct keys with sorted counters/probability for right pairs (Final)
  print("Top Key Counter")
  for i in range(0,2**(key_size)):                                                # How many times a key is with highest counter in the given set of right pairs 
    print(str(bin(i))[2:].zfill(4) + " : " + str(counter_top[i]) )

Key is:  0000
{'00000000000001101010': 250, '00000000000001101011': 242, '00000000000010111111': 5, '00000000000011111101': 5, '00000000000001100000': 5, '00000000000010011101': 5, '00000000000011010100': 5, '00000000000011100001': 5, '00000000000001101101': 5, '00000000000000100110': 5, '00000000000000101001': 5, '00000000000001111010': 5, '00000000000011011001': 5, '00000000000011000101': 5, '00000000000011010111': 5, '00000000000011111000': 5, '00000000000011110000': 5, '00000000000010110110': 4, '00000000000010001010': 4, '00000000000001001011': 4, '00000000000001010011': 4, '00000000000010100100': 4, '00000000000000111111': 4, '00000000000001000111': 4, '00000000000011100100': 4, '00000000000000011111': 4, '00000000000000101110': 4, '00000000000011101101': 4, '00000000000011111001': 4, '00000000000001000001': 4, '00000000000000111010': 4, '00000000000000101010': 4, '00000000000001100011': 4, '00000000000011010010': 4, '00000000000001101110': 4, '00000000000011111100': 4, '00000000